In [1]:

from components.controller.bluetooth_controller import BlueToothCarControlSPP
from components.controller.bluetooth_SPP_server import start_bluetooth_server
from components.gyroscope.gyroscope import AngularSpeedControlV2
from components.two_wheels import TwoWheelsV2
from components.camera import PicameraV2
from components.controller.image_ml import ImageMLController
from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager

from components import default_loop

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)
from ctypes import c_double



%load_ext autoreload
%autoreload 2

In [2]:
logger_set = LoggerSet('../log/test0', overwrite_ok=False)
manager = Manager()

#speed = manager.Value(c_double, 0)
#angular_velocity = manager.Value(c_double, 0)


In [3]:
_, _, w_starter = TwoWheelsV2.create_component(
    init_kwargs= dict(
        left_pin=33, 
        right_pin=32, 
        dir_pins=(16, 18), 
        logger_set=logger_set, 
        name='TwoWheelsV2'
    ), 
    mainloop=default_loop, 
    main_kwargs=dict(interval=1/50), 
    manager=manager, 
)

imu_out, _, imu_starter = AngularSpeedControlV2.create_component(
    init_kwargs= dict(
        logger_set=logger_set, 
        name='AngularSpeedControlV2'
    ), 
    mainloop=default_loop, 
    main_kwargs=dict(interval=1/50), 
    manager=manager, 
)

bt_out, _, bt_starter = BlueToothCarControlSPP.create_component(
    init_kwargs= dict(
        logger_set=logger_set, 
        name='BlueToothCarControlSPP'
    ), 
    mainloop=default_loop, 
    main_kwargs=dict(interval=1/30), 
    manager=manager, 
)

cam_out, _, cam_starter, cam_out_reader = PicameraV2.create_camera_component(
    init_kwargs= dict(
        resolution=(114, 64), framerate=30,
        logger_set=logger_set, 
        name='PicameraV2'
    ), 
    mainloop=default_loop, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)

ml_out, _, ml_starter = ImageMLController.create_component(
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
    proxy_reader=cam_out_reader, 
)

bt_ser_out, bt_ser = start_bluetooth_server(manager)


Making discoverable...
Starting Serial Port Profile...
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 86)


In [4]:
bt = bt_starter(bt_ser_out)
imu = imu_starter(bt_out)
wheel = w_starter(imu_out)
cam = cam_starter()
ml = ml_starter(cam_out)

[0:12:06.011608924] [3987]  INFO Camera camera_manager.cpp:297 libcamera v0.0.5+83-bde9b04f
[0:12:06.073623940] [4009]  INFO RPI vc4.cpp:437 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media0
[0:12:06.073979621] [4009]  INFO RPI pipeline_base.cpp:1101 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:12:06.219610591] [3987]  INFO Camera camera.cpp:1033 configuring streams: (0) 114x64-RGB888 (1) 1920x1080-SGRBG10_CSI2P
[0:12:06.220926407] [4009]  INFO RPI vc4.cpp:565 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected unicam format: 1920x1080-pgAA
/home/kawa/projects/car1/.venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 30 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
bt.terminate()
imu.terminate()
wheel.terminate()
cam.terminate()
ml.terminate()


In [5]:
imu = imu_starter(bt_out)
wheel = w_starter(imu_out)

In [80]:
import time
def timeout_callback(func, sec, check_interval=0.1, *args, **kwargs):
    start = time.monotonic()
    while (time.monotonic() - start) < sec:
        time.sleep(check_interval)
    func(*args, **kwargs)

speed.value=100

def f(x):
    x.value=0
timeout_callback(f, 4, 0.1, speed)

In [5]:
asdasd, _, cam_starter = PicameraV2.create(
    manager, 
    interval=1/30, 
    entry_kwargs=dict(
        resolution=(114, 64), framerate=30,
        logger_set=logger_set, 
        name='PicameraV2'
    )
)



In [6]:
asdasd

[None]